In [1]:
## Pre-setting
# automatically adjust the width of the notebook code cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# if one module is changed, this line will automatically reload that module
%load_ext autoreload
%autoreload 2

# display the figure in the notebook
%matplotlib inline

# To change the font size in acrobat
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

In [2]:
## Add path
import os
import sys
src_dir = os.path.abspath(os.path.join(os.pardir, 'src'))
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

In [3]:
## Private package

from corpora.pp_single_stopword import synthetic_single_stopword_terminal

from models.modelfront import topicmodel_inference_front

from measures.overlap import obtain_nmi_acc_unsup

# creat synthetic benchmark data

In [4]:
## Parameters for distribution

V = 400 # nubmer of words
K = 5 # nubmer of topics

dist_w = 'uni'  # global word frequency. uni or zipf
# dist_t = dist_w # number of words in each topic. uni or zipf

dist_stop = 'uni' # probability of stopwords appearance.
p_s = 0.1 # 100 * p_s is the percentage of the stopword 

c_w = 0.8  # degree of structure for word mixing for each topic. 0: random; 1: structure
# c_t = c_w # degree of structure for topic mixing for each document. 0: random; 1: structure

## Parameters for document

D = 2000 # nubmer of document
m = 100 # length of document


## Parameters for random function
seed = 5
burstiness = None


In [5]:
%%time
dict_out_syn_stop = synthetic_single_stopword_terminal(V = V , K = K, D = D, m = m, dist_w =dist_w, dist_stop = dist_stop, p_s = p_s, c_w = c_w, seed = seed, burstiness = burstiness)

CPU times: user 1.65 s, sys: 31.3 ms, total: 1.68 s
Wall time: 1.68 s


In [6]:
dict_out_syn_stop.keys()

dict_keys(['n_jd', 'n_wj', 'p_wt', 'p_w_td', 'p_td', 'n_wd', 'texts', 'V_t', 'p_t', 'p_w', 'document_topic_assign_list', 'state_dwz', 'word_topic_assign_list'])

In [7]:
texts = dict_out_syn_stop['texts']


In [8]:
topic_list_ture = dict_out_syn_stop['document_topic_assign_list']
len(topic_list_ture)

2000

## run topic model: ldavb

In [9]:

dict_input_ldavb = {
    
    ## choose topic model
    'topic_model': 'ldavb'
    
    ## provide corpus and number of topics if need
    , 'texts':texts
    , 'input_k': K

    ## optional, only works for synthetic corpus with token labeling
#     , 'state_dwz_true': state_dwz
#     , 'k_true': K 
#     , 'input_v': V  # only need for ldavb- token labeling
    
    ## optional
    , 'dN_opt':0 ## optional
    , 'minimum_probability':0 ## optional   
    }

In [10]:
%%time

dict_output_ldavb = topicmodel_inference_front( dict_input_ldavb )

CPU times: user 12.6 s, sys: 72.8 ms, total: 12.7 s
Wall time: 12.7 s


In [11]:
dict_output_ldavb.keys()

dict_keys(['state_dwz_infer', 'p_wt_infer', 'p_td_infer'])

In [12]:
p_td_infer = dict_output_ldavb['p_td_infer']

## unsupervised document classification by NMI

In [13]:
nmi_unsup, acc_unsup = obtain_nmi_acc_unsup(topic_list_ture, p_td_infer)

In [14]:
nmi_unsup

0.75980530240166932